In [1]:
import pandas as pd
import numpy as np
from datetime import *
import time

In [2]:
path = '/Users/panbingqing/Documents/工作/城市大脑/杭州/卡口/卡口数据/'
city_data = pd.read_csv(path + '20180407.csv')
#city_data = pd.read_csv(path + '2018-04-22.csv')
kakou_info = pd.read_csv('/Users/panbingqing/Documents/工作/城市大脑/杭州/卡口/kakou_area.csv')
data = pd.DataFrame()

In [3]:
new_data = pd.merge(city_data,kakou_info,on='devc_name')[['dev_id','devc_name','vhc_no','spot_time','loc','spot_direction','in_area']]
new_data = new_data.replace('0',np.NaN) #去除0值

In [111]:
#求当前时刻的各个交通指标
data = pd.DataFrame()
time_now = '2018-04-07 00:02:00'
time_list = []
area1 = ['上城区','江干区','西湖区','拱墅区','下城区','滨江区']
area = []
#area = ['桐庐县', '临安区', '下城区','富阳区','上城区','拱墅区','西湖区','江干区','滨江区','余杭区','萧山区','建德市']  
for i in range(0,720):
    #在途量
    last_time = str(datetime.strptime(time_now,'%Y-%m-%d %X') - timedelta(hours=1))
    way_vhc = new_data[(new_data.spot_time >= last_time) & (new_data.spot_time < time_now)].sort_values(
                'spot_time').drop_duplicates('vhc_no',keep='last').reset_index(drop=True).groupby('in_area').count()['vhc_no']  

    #交通量
    acc_vhc = new_data[new_data.spot_time <= time_now].drop_duplicates('vhc_no').groupby('in_area').vhc_no.count()

    #出入城量
    last_time1 = str(datetime.strptime(time_now,'%Y-%m-%d %X') - timedelta(minutes=61))
    time_now1 = str(datetime.strptime(time_now,'%Y-%m-%d %X') + timedelta(minutes=1))
    data_inline1 = new_data[(new_data.spot_time >= last_time1) & (new_data.spot_time < time_now1)].sort_values('spot_time').drop_duplicates('vhc_no',keep='last').reset_index(drop=True)
    data_inline2 = new_data[(new_data.spot_time >= last_time) & (new_data.spot_time < time_now)].sort_values('spot_time').drop_duplicates('vhc_no',keep='last').reset_index(drop=True)

    data_inline = pd.merge(data_inline1,data_inline2[['vhc_no','in_area']],on = 'vhc_no')
    data_inline = data_inline[data_inline.in_area_x != data_inline.in_area_y] #去除在同一个区域的
    #筛选入城区域和出城区域
    data_inline['start_city'] = data_inline.in_area_x.apply(lambda x : x in area1) #在城内
    data_inline['over_city'] = data_inline.in_area_y.apply(lambda x : x not in area1)  #在城外
    data_inline_out = data_inline[(data_inline['start_city'] == True) & (data_inline['over_city'] == True)].groupby('over_city').count()['vhc_no'] #出城
    data_inline_in = data_inline[(data_inline['start_city'] == False) & (data_inline['over_city'] == False)].groupby('start_city').count()['vhc_no'] #入城

    #驻车量
    traffic = new_data[new_data.spot_time <= time_now].drop_duplicates('vhc_no') #交通量
    way = new_data[(new_data.spot_time >= last_time) & (new_data.spot_time < time_now)].sort_values(
                'spot_time').drop_duplicates('vhc_no',keep='last').reset_index(drop=True)

    wait_vhc = traffic[~traffic.vhc_no.isin(way.vhc_no)].groupby('in_area').vhc_no.count() #停车量

    new = pd.DataFrame(
        {'time':time_now,
         'acc_vhc':acc_vhc,
         'way_vhc':way_vhc,
         'in_city':data_inline_in,
         'out_city':data_inline_out,
         'wait_vhc':wait_vhc 
    },index=acc_vhc.index)
    #area.append(acc_vhc.index)
    data = data.append(new)

    time_now = str(datetime.strptime(time_now,'%Y-%m-%d %X')+ timedelta(minutes=2))   

In [110]:
data_inline_out

over_city
True    106
Name: vhc_no, dtype: int64

In [102]:
data.to_csv(path + 'data_new_0407.csv')

In [45]:
new

,time,area,acc_vhc,way_vhc,in_city,out_city,wait_vhc
上城区,2018-04-07 00:02:00,上城区,281,278,NaN,NaN,2.0
下城区,2018-04-07 00:02:00,下城区,298,297,NaN,NaN,2.0
余杭区,2018-04-07 00:02:00,余杭区,7,8,NaN,NaN,NaN
拱墅区,2018-04-07 00:02:00,拱墅区,243,242,NaN,NaN,1.0
江干区,2018-04-07 00:02:00,江干区,273,270,NaN,NaN,NaN
滨江区,2018-04-07 00:02:00,滨江区,34,33,NaN,NaN,NaN
萧山区,2018-04-07 00:02:00,萧山区,86,86,NaN,NaN,NaN
西湖区,2018-04-07 00:02:00,西湖区,182,185,NaN,NaN,NaN


""


In [ ]:
data.append(new)

In [ ]:
data.append(new,ignore_index=True)